In [13]:
import sys
import json
import numpy as np
import pandas as pd

In [15]:
with open('recipe_export.json', 'r') as f:
    data = json.load(f)

JSONDecodeError: Extra data: line 2 column 1 (char 11889)

In [20]:
data = pd.read_csv('recipe_export.csv', engine=None)

ParserError: Error tokenizing data. C error: Expected 434 fields in line 54, saw 440


In [ ]:
ObjectId("5e6ad1aec952194b1ce2c15b"),http://www.edamam.com/ontologies/edamam.owl#recipe_5d45496e26d92dad19af67816d9d8ac5,Spinach Souffle,https://www.edamam.com/web-img/7bf/7bf4d5f9c9a7c8945a5a163a3a8dcbf2.jpg,Cookstr,http://www.cookstr.com/recipes/spinach-souffle-julia-child,"["Low-Carb"]","["Sugar-Conscious","Vegetarian","Peanut-Free","Tree-Nut-Free","Alcohol-Free"]","["Sulfites"]","["1 Tb minced shallots or green onion","1 Tb butter","¾ cup blanched chopped spinach (or chopped frozen spinach—which will take several minutes more cooking)","¼ tsp salt","5 egg whites","A pinch of salt","1/3 to ½ cup (1½ to 2 ounces) grated Swiss cheese","1 tsp butter","1 Tb grated Swiss or Parmesan cheese","3 Tb butter","3 Tb flour","1 cup boiling milk","1/2 tsp salt","1/8 tsp pepper","A pinch of cayenne pepper","Pinch of nutmeg","4 egg yolks"]",1171.718720449178,610.1684290695503,60,"{"ENERC_KCAL":{"label":"Energy","quantity":1171.718720449178,"unit":"kcal"},"FAT":{"label":"Fat","quantity":90.28297950057123,"unit":"g"},"FASAT":{"label":"Saturated","quantity":52.55907196219361,"unit":"g"},"FATRN":{"label":"Trans","quantity":2.016926083341196,"unit":"g"},"FAMS":{"label":"Monounsaturated","quantity":25.956585337919652,"unit":"g"},"FAPU":{"label":"Polyunsaturated","quantity":5.355751838362568,"unit":"g"},"CHOCDF":{"label":"Carbs","quantity":37.4728756853422,"unit":"g"},"FIBTG":{"label":"Fiber","quantity":1.528258333361611,"unit":"g"},"SUGAR":{"label":"Sugars","quantity":14.716876484192177,"unit":"g"},"PROCNT":{"label":"Protein","quantity":52.8446419273731,"unit":"g"},"CHOLE":{"label":"Cholesterol","quantity":796.4261738984324,"unit":"mg"},"NA":{"label":"Sodium","quantity":1414.0856272143408,"unit":"mg"},"CA":{"label":"Calcium","quantity":962.6857304556735,"unit":"mg"},"MG":{"label":"Magnesium","quantity":93.65840051439321,"unit":"mg"},"K":{"label":"Potassium","quantity":843.7142234992883,"unit":"mg"},"FE":{"label":"Iron","quantity":2.9315468174878134,"unit":"mg"},"ZN":{"label":"Zinc","quantity":5.1171349613730746,"unit":"mg"},"P":{"label":"Phosphorus","quantity":870.0737182078414,"unit":"mg"},"VITA_RAE":{"label":"Vitamin A","quantity":994.8244306995668,"unit":"µg"},"VITC":{"label":"Vitamin C","quantity":7.798675000005622,"unit":"mg"},"THIA":{"label":"Thiamin (B1)","quantity":0.3035344242454973,"unit":"mg"},"RIBF":{"label":"Riboflavin (B2)","quantity":1.5526751714541966,"unit":"mg"},"NIA":{"label":"Niacin (B3)","quantity":1.0111617822288317,"unit":"mg"},"VITB6A":{"label":"Vitamin B6","quantity":0.40906139900586186,"unit":"mg"},"FOLDFE":{"label":"Folate equivalent (total)","quantity":156.17371659483828,"unit":"µg"},"FOLFD":{"label":"Folate (food)","quantity":156.17371659483828,"unit":"µg"},"VITB12":{"label":"Vitamin B12","quantity":4.198989209989993,"unit":"µg"},"VITD":{"label":"Vitamin D","quantity":291.4033664272273,"unit":"IU"},"TOCPHA":{"label":"Vitamin E","quantity":3.8495744954532434,"unit":"mg"},"VITK1":{"label":"Vitamin K","quantity":128.9950349700677,"unit":"µg"},"WATER":{"label":"Water","quantity":421.2395221658756,"unit":"g"}}","{"ENERC_KCAL":{"label":"Energy","quantity":58.585936022458895,"unit":"%"},"FAT":{"label":"Fat","quantity":138.89689153934037,"unit":"%"},"FASAT":{"label":"Saturated","quantity":262.7953598109681,"unit":"%"},"CHOCDF":{"label":"Carbs","quantity":12.490958561780733,"unit":"%"},"FIBTG":{"label":"Fiber","quantity":6.113033333446444,"unit":"%"},"PROCNT":{"label":"Protein","quantity":105.6892838547462,"unit":"%"},"CHOLE":{"label":"Cholesterol","quantity":265.47539129947745,"unit":"%"},"NA":{"label":"Sodium","quantity":58.9202344672642,"unit":"%"},"CA":{"label":"Calcium","quantity":96.26857304556735,"unit":"%"},"MG":{"label":"Magnesium","quantity":22.299619170093624,"unit":"%"},"K":{"label":"Potassium","quantity":17.951366457431668,"unit":"%"},"FE":{"label":"Iron","quantity":16.28637120826563,"unit":"%"},"ZN":{"label":"Zinc","quantity":46.51940873975522,"unit":"%"},"P":{"label":"Phosphorus","quantity":124.29624545826304,"unit":"%"},"VITA_RAE":{"label":"Vitamin A","quantity":110.53604785550743,"unit":"%"},"VITC":{"label":"Vitamin C","quantity":8.665194444450691,"unit":"%"},"THIA":{"label":"Thiamin (B1)","quantity":25.294535353791442,"unit":"%"},"RIBF":{"label":"Riboflavin (B2)","quantity":119.43655165032283,"unit":"%"},"NIA":{"label":"Niacin (B3)","quantity":6.319761138930198,"unit":"%"},"VITB6A":{"label":"Vitamin B6","quantity":31.46626146198937,"unit":"%"},"FOLDFE":{"label":"Folate equivalent (total)","quantity":39.04342914870957,"unit":"%"},"VITB12":{"label":"Vitamin B12","quantity":174.95788374958303,"unit":"%"},"VITD":{"label":"Vitamin D","quantity":1942.6891095148487,"unit":"%"},"TOCPHA":{"label":"Vitamin E","quantity":25.663829969688287,"unit":"%"},"VITK1":{"label":"Vitamin K","quantity":107.49586247505641,"unit":"%"}}","[{"label":"Fat","tag":"FAT","schemaOrgTag":"fatContent","total":90.28297950057123,"hasRDI":true,"daily":138.89689153934037,"unit":"g","sub":[{"label":"Saturated","tag":"FASAT","schemaOrgTag":"saturatedFatContent","total":52.55907196219361,"hasRDI":true,"daily":262.7953598109681,"unit":"g"},{"label":"Trans","tag":"FATRN","schemaOrgTag":"transFatContent","total":2.016926083341196,"hasRDI":false,"daily":0,"unit":"g"},{"label":"Monounsaturated","tag":"FAMS","schemaOrgTag":null,"total":25.956585337919652,"hasRDI":false,"daily":0,"unit":"g"},{"label":"Polyunsaturated","tag":"FAPU","schemaOrgTag":null,"total":5.355751838362568,"hasRDI":false,"daily":0,"unit":"g"}]},{"label":"Carbs","tag":"CHOCDF","schemaOrgTag":"carbohydrateContent","total":37.4728756853422,"hasRDI":true,"daily":12.490958561780733,"unit":"g","sub":[{"label":"Carbs (net)","tag":"CHOCDF.net","schemaOrgTag":null,"total":35.94461735198059,"hasRDI":false,"daily":0,"unit":"g"},{"label":"Fiber","tag":"FIBTG","schemaOrgTag":"fiberContent","total":1.528258333361611,"hasRDI":true,"daily":6.113033333446444,"unit":"g"},{"label":"Sugars","tag":"SUGAR","schemaOrgTag":"sugarContent","total":14.716876484192177,"hasRDI":false,"daily":0,"unit":"g"},{"label":"Sugars, added","tag":"SUGAR.added","schemaOrgTag":null,"total":0,"hasRDI":false,"daily":0,"unit":"g"}]},{"label":"Protein","tag":"PROCNT","schemaOrgTag":"proteinContent","total":52.8446419273731,"hasRDI":true,"daily":105.6892838547462,"unit":"g"},{"label":"Cholesterol","tag":"CHOLE","schemaOrgTag":"cholesterolContent","total":796.4261738984324,"hasRDI":true,"daily":265.47539129947745,"unit":"mg"},{"label":"Sodium","tag":"NA","schemaOrgTag":"sodiumContent","total":1414.0856272143408,"hasRDI":true,"daily":58.9202344672642,"unit":"mg"},{"label":"Calcium","tag":"CA","schemaOrgTag":null,"total":962.6857304556735,"hasRDI":true,"daily":96.26857304556735,"unit":"mg"},{"label":"Magnesium","tag":"MG","schemaOrgTag":null,"total":93.65840051439321,"hasRDI":true,"daily":22.299619170093624,"unit":"mg"},{"label":"Potassium","tag":"K","schemaOrgTag":null,"total":843.7142234992883,"hasRDI":true,"daily":17.951366457431668,"unit":"mg"},{"label":"Iron","tag":"FE","schemaOrgTag":null,"total":2.9315468174878134,"hasRDI":true,"daily":16.28637120826563,"unit":"mg"},{"label":"Zinc","tag":"ZN","schemaOrgTag":null,"total":5.1171349613730746,"hasRDI":true,"daily":46.51940873975522,"unit":"mg"},{"label":"Phosphorus","tag":"P","schemaOrgTag":null,"total":870.0737182078414,"hasRDI":true,"daily":124.29624545826304,"unit":"mg"},{"label":"Vitamin A","tag":"VITA_RAE","schemaOrgTag":null,"total":994.8244306995668,"hasRDI":true,"daily":110.53604785550743,"unit":"µg"},{"label":"Vitamin C","tag":"VITC","schemaOrgTag":null,"total":7.798675000005622,"hasRDI":true,"daily":8.665194444450691,"unit":"mg"},{"label":"Thiamin (B1)","tag":"THIA","schemaOrgTag":null,"total":0.3035344242454973,"hasRDI":true,"daily":25.294535353791442,"unit":"mg"},{"label":"Riboflavin (B2)","tag":"RIBF","schemaOrgTag":null,"total":1.5526751714541966,"hasRDI":true,"daily":119.43655165032283,"unit":"mg"},{"label":"Niacin (B3)","tag":"NIA","schemaOrgTag":null,"total":1.0111617822288317,"hasRDI":true,"daily":6.319761138930198,"unit":"mg"},{"label":"Vitamin B6","tag":"VITB6A","schemaOrgTag":null,"total":0.40906139900586186,"hasRDI":true,"daily":31.46626146198937,"unit":"mg"},{"label":"Folate equivalent (total)","tag":"FOLDFE","schemaOrgTag":null,"total":156.17371659483828,"hasRDI":true,"daily":39.04342914870957,"unit":"µg"},{"label":"Folate (food)","tag":"FOLFD","schemaOrgTag":null,"total":156.17371659483828,"hasRDI":false,"daily":0,"unit":"µg"},{"label":"Folic acid","tag":"FOLAC","schemaOrgTag":null,"total":0,"hasRDI":false,"daily":0,"unit":"µg"},{"label":"Vitamin B12","tag":"VITB12","schemaOrgTag":null,"total":4.198989209989993,"hasRDI":true,"daily":174.95788374958303,"unit":"µg"},{"label":"Vitamin D","tag":"VITD","schemaOrgTag":null,"total":291.4033664272273,"hasRDI":true,"daily":1942.6891095148487,"unit":"µg"},{"label":"Vitamin E","tag":"TOCPHA","schemaOrgTag":null,"total":3.8495744954532434,"hasRDI":true,"daily":25.663829969688287,"unit":"mg"},{"label":"Vitamin K","tag":"VITK1","schemaOrgTag":null,"total":128.9950349700677,"hasRDI":true,"daily":107.49586247505641,"unit":"µg"},{"label":"Sugar alcohols","tag":"Sugar.alcohol","schemaOrgTag":null,"total":0,"hasRDI":false,"daily":0,"unit":"g"},{"label":"Water","tag":"WATER","schemaOrgTag":null,"total":421.2395221658756,"hasRDI":false,"daily":0,"unit":"g"}]"
